In [99]:
# Do Minh Canh - s1920426
import re
import numpy as np
import matplotlib.pyplot as plt

In [100]:
"""
Nonterminal symbol
"""
class Nonterminal():
    def __init__(self, symbol):
        self._symbol = symbol
        self._hash = hash(symbol)

    def symbol(self):
        return self._symbol
    
    def __eq__(self, other):
        return type(self) == type(other) and self._symbol == other._symbol
    
    def __hash__(self):
        return self._hash
    
    def __repr__(self):
        return "%s" % self._symbol
    
def is_terminal(item):
    return not isinstance(item, Nonterminal)

def is_nonterminal(item):
    return isinstance(item, Nonterminal)

"""
Production or Rule
- lhs is a nonterminal symbol
- rhs is a tuple
"""
class Production():
    def __init__(self, lhs, rhs):
        self._lhs = lhs
        self._rhs = tuple(rhs)
        assert is_nonterminal(lhs), 'Left hand side must be a nonterminal'
        self._hash = hash((self._lhs, self._rhs))
    
    def lhs(self):
        return self._lhs
    
    def rhs(self):
        return self._rhs
    
    def is_nonlexical(self):
        """
        Return True if the righ-hand contains only nonterminal token.
        """
        return all(is_nonterminal(n) for n in self._rhs)
    
    def is_lexical(self):
        """
        return True if the right-hand side contains at least on terminal token
        """
        return not self.is_nonlexical()
    
    def __hash__(self):
        return self._hash
    
    def __eq__(self, other):
        return (
            type(self) == type(other) and
            self._lhs == other._lhs and
            self._rhs == other._rhs
        )
    
    def __repr__(self):
        result = "%s -> " % repr(self._lhs)
        result += " ".join(repr(el) for el in self._rhs)
        return result

In [101]:
def is_terminal_text(item):
    return re.search('^[A-Z]', item) is None

def is_nonterminal_text(item):
    return not re.search('^[A-Z]', item) is None

"""
Context Free Grammar - CFG
- read FFG from a list of strings that presents for rules
"""
class CFG():
    def __init__(self, lines):
        self._rules = set()
        self._start = None
        for idx, line in enumerate(lines):
            tokens = line.split()
            assert len(tokens) > 1, '% is invalid' % line
            assert is_nonterminal_text(tokens[0]), 'lhs must be a nonterminal'
            lhs = Nonterminal(tokens[0])
            rhs = []
            if idx == 0: self._start = lhs
            for token in tokens[1:]:
                if is_nonterminal_text(token):
                    rhs.append(Nonterminal(token))
                else:
                    rhs.append(token)

            if all(is_terminal(n) for n in rhs):
                # if the lhs contains only terminal tokens, we should make it separated
                for token in rhs:
                    self._rules.add(Production(lhs, token))
            else:
                self._rules.add(Production(lhs, rhs))
    
    def start(self):
        return self._start
    
    def rules(self):
        return self._rules
    
    def addRule(self, rule):
        self._rules.add(rule)
        
    def getNonLexicalRule(self):
        for rule in self._rules:
            if rule.is_nonlexical() and len(rule.rhs()) > 2:
                return rule
        return None
    
    def findRule(self, rhs):
        # find rule with exactly same rhs
        rules = set()
        for rule in self.rules():
            if rule.rhs() == tuple(rhs):
                rules.add(rule)
        return rules
    
    def findRuleIn(self, eleSet):
        # find rule such that lhs is not in eleSet and 
        # rhs intersects with eleSet that is not empty
        rules = set()
        for rule in self.rules():
            if (rule.lhs() not in eleSet) and len(set(rule.rhs()) & eleSet) > 0:
                rules.add(rule)
        return rules
    
    def toFile(self, path):
        f = open(path, "w")
        for rule in self.rules():
            f.write("%s\t%s\n" % (rule.lhs().symbol(), "\t".join(repr(el) for el in rule.rhs())))
        f.close()
        
    def __repr__(self):
        result = "Start symbol is %s\n" % repr(self._start)
        result += "\n".join(repr(rule) for rule in self._rules)
        return result

In [102]:
"""
Used to build a combination from choices
"""
class Combination():
    def __init__(self, choices):
        self.solutions = []
        self.visited = [0] * len(choices)
        self.choices = choices
        
    def build(self):
        if len(self.solutions) == 0:
            self.backtrack(0)
        return self.solutions
    
    def addSolution(self):
        solution = []
        for idx, x in enumerate(self.choices):
            if (self.visited[idx] == 1):
                solution.append(x)
        self.solutions.append(solution)
        
    def backtrack(self, i):
        if (i >= len(self.choices)):
            self.addSolution()
            return
        self.visited[i] = 0
        self.backtrack(i + 1)
        self.visited[i] = 1
        self.backtrack(i + 1)
        
"""
Chomsky Normal Form (CNF) - extends from CFG
- Reading from a list of strings that presents rules. Then such rules are converted to CNF
"""
class CNF(CFG):
    
    def __init__(self, lines, convertCNF=True):
        super().__init__(lines)
        if convertCNF:
            self.convertToCNF()
            
    def convertToCNF(self):
        self.eliminateStart()
        self.simplifyCFG()
        self.removeMixedRules()
        self.makeRulesBinary()
    
    def eliminateStart(self):
        start = self.start()
        need_to_add = None
        for rule in self.rules():
            if start in rule.rhs():
                need_to_add = True
                break
        if need_to_add:
            self._start = Nonterminal('S*')
            self.addRule(Production(self.start(), [start]))
            
    def simplifyCFG(self):
        self.removeUselessProduction()
        self.removeNullProduction()
        self.removeUnitProductions()
        
    def removeUselessProduction(self):
        uselessRules = set()
        start = self.start()
        for rule in self.rules():
            if rule.lhs() == start:
                continue
            used = any([rule.lhs() in r.rhs() for r in self.rules() if r != rule])
            if not used:
                uselessRules.add(rule)
        self._rules = self.rules() - uselessRules
    
    def removeNullProduction(self):
        # assuming that 'λ' is used the nul production
        # finding nul variables set
        nulVarSet = set(['λ'])
        while True:
            rules = self.findRuleIn(nulVarSet)
            if len(rules) == 0:
                break
            nulVarSet = nulVarSet | {rule.lhs() for rule in rules}
        nulVarSet = nulVarSet  - set(['λ', self.start()])
        
        # make new rules with null variables set by combination
        rules = set()
        for rule in self.rules():
            if len(rule.rhs()) == 1 and rule.rhs()[0] == 'λ':
                continue
            inter = set(rule.rhs()) & nulVarSet
            if len(inter) == 0:
                rules.add(rule)
            if len(inter) > 0:
                rhs = rule.rhs()
                # get combination of intersected elements
                cbi = Combination(list(inter)).build()
                # create a new rule for each combination
                for sol in cbi:
                    new_rhs = []
                    for ele in rhs:
                        if ele not in sol:
                            new_rhs.append(ele)
                    if len(new_rhs) > 0:
                        rules.add(Production(rule.lhs(), new_rhs))
        self._rules = rules
    
    def removeUnitProductions(self):
        rules = {rule for rule in self.rules() if \
                 len(rule.rhs()) > 1 or \
                 (len(rule.rhs()) == 1 and is_terminal(rule.rhs()[0])) \
                 }
        unit_rules = self.rules() - rules
        
        for unit_rule in unit_rules:
            values = set()
            single_variable = unit_rule.rhs()[0]
            for rule in rules:
                if rule.lhs() == single_variable:
                    values.add(rule.rhs())
            
            for val in values:
                rules.add(Production(unit_rule.lhs(), val))
        self._rules = rules
        self.removeUselessProduction()
        
    def removeMixedRules(self, padding='_'):
        # finding lexical rules
        lexical_rules = {rule for rule in self.rules() if rule.is_lexical() and len(rule.rhs()) > 1}
        add_rules = set()
        for lexical_rule in lexical_rules:
            rhs = []
            for token in lexical_rule.rhs():
                if is_terminal(token):
                    new_token = Nonterminal(token + padding)
                    add_rules.add(Production(new_token, token))
                    rhs.append(new_token)
                else:
                    rhs.append(token)
            add_rules.add(Production(lexical_rule.lhs(), rhs))
        self._rules = (self.rules() - lexical_rules) | add_rules
    
    def makeRulesBinary(self, padding='&'):
        while True:
            rule = self.getNonLexicalRule()
            if rule is None:
                break;
            
            lhs = rule.lhs()
            rhs = list(rule.rhs())
            head = rhs[:2]
            remainder = rhs[2:] 
            
            new_token = Nonterminal(head[0].symbol() + padding + head[1].symbol())
            self._rules.add(Production(lhs, [new_token] + remainder))
            self._rules.add(Production(new_token, head))
            self._rules.remove(rule)

In [103]:
"""
Each cell data structure in the CKY table
"""
class CKY_Cell():
    def __init__(self, i, j):
        self._solutions = []
        self._position = (i, j)

    def addSolution(self, rule, left=None, right=None, idxLeft=None, idxRight=None):
        self._solutions.append((rule, left, right, idxLeft, idxRight))
    
    def solutions(self):
        return self._solutions
    
    def __repr__(self):
        if len(self._solutions) == 0:
            return '∅'
        else:
            results = []
            for (rule, left, right, idxLeft, idxRight) in self._solutions:
                if not isinstance(left, CKY_Cell) or not isinstance(right, CKY_Cell):
                    results.append(rule.lhs().symbol())
                else:
                    results.append('%s { %s , %s }' % (rule.lhs().symbol(),\
                                                  left.solutions()[idxLeft][0].lhs().symbol(),\
                                                 right.solutions()[idxRight][0].lhs().symbol()))
                
            return "\n".join(results)

"""
CKY algorithm given an grammar in the form of CNF
"""
class CKY():
    def __init__(self, grammar):
        self._grammar = grammar
    
    """
    Parse a sentence and return (tokens, matrix)
    - tokens is built from the sentence
    - matrix is behalf on the CKY table
    """
    def parse(self, sentence):
        grammar = self.grammar()
        tokens = [c for c in sentence if c != ' ']
        N = len(tokens)
        M = np.empty((N, N), dtype=object)
        # initialization
        for i in range(N):
            cell = CKY_Cell(i, i)
            rules = grammar.findRule([tokens[i]])
            assert len(rules) > 0, 'Data is invalid'
            for rule in rules:
                if rule.lhs() != grammar.start():
                    cell.addSolution(rule)
            M[i][i] = cell

        # start building CKY table
        for d in range(1, N):
            for i in range(N - d):
                j = i + d
                cell = CKY_Cell(i, j)
                for k in range(i, j):
                    if not isinstance(M[i][k], CKY_Cell) or not isinstance(M[k+1][j], CKY_Cell):
                        continue
                    leftSolutions = M[i][k].solutions()
                    rightSolutions = M[k+1][j].solutions()
                    for idxLeft, (leftRule, *restLef) in enumerate(leftSolutions):
                        for idxRight, (rightRule, *restRight) in enumerate(rightSolutions):
                            rules = grammar.findRule([leftRule.lhs(), rightRule.lhs()])
                            for rule in rules:
                                if rule.lhs() != grammar.start() or (i == 0 and j == N-1):
                                    cell.addSolution(rule, M[i][k], M[k+1][j], idxLeft, idxRight)
                # if len(cell.solutions()) > 0:
                M[i][j] = cell
        count = 0
        if isinstance(M[0][N-1], CKY_Cell):
            for rule, *rest in M[0][N-1].solutions():
                if rule.lhs() == grammar.start():
                    count += 1
                    
        if count == 0:
            print("Cannot parse %s" % sentence)
        else:
            print('Parse sucessfully. Number of parse tree is %d' % count)
        return tokens, M
    
    def grammar(self):
        return self._grammar

In [104]:
"""
Drawing a table given a name of output file, columns and matrix values
Output is an image file
"""
def draw_table(fileName, columns, values):
    fig, ax = plt.subplots()
    ax.set_axis_off()
    ytable = ax.table(cellText=values,
                      colLabels=columns, 
                      colColours =["palegreen"]*len(columns),
                      cellLoc='center', 
                      loc='upper left', colWidths=[1]*len(columns))
    ytable.set_fontsize(20)
    ytable.scale(1, 10)
    cellDict = ytable.get_celld()
    for i in range(0, len(columns)):
        cellDict[(0, i)].set_text_props(color='b', fontweight='bold')
    plt.savefig(fileName, bbox_inches='tight')
    plt.close()

In [105]:
"""
Synthesizing what need to do in mini project 3
Input:
    - input/cfg_rule.txt file
    - input/sentence.txt file
Ouput:
    - output/cnf_rule.txt
    - output/CKY_Table.png
    - output/Filled_CKY_Table.png
"""
def project3():
    # reading cfg from file
    cfg_file = open('input/cfg_rule.txt', 'r')
    cfg_lines = cfg_file.readlines()
    # reading the sentence from file
    sentence_file = open('input/sentence.txt', 'r')
    sentence = sentence_file.readline()
    # building CNF grammar
    cnf = CNF(cfg_lines)
    # save CNF grammar to file
    cnf.toFile("output/cnf_rule.txt")
    # using CKY with the CNF grammar
    cky = CKY(cnf)
    # parsing a sentense
    columns, matrix = cky.parse(sentence)
    # draw CKY table for the sentence
    draw_table('output/CKY_Table.png', columns, np.empty((len(columns), len(columns)), dtype=object))
    # draw CKY table with filled values by the CKY algorithm
    draw_table('output/Filled_CKY_Table.png', columns, matrix)

In [106]:
"""
Let's see the result in the `output` folder
"""
project3()

Parse sucessfully. Number of parse tree is 1
